In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import requests
import folium
# !pip install pandas # 파이썬 데이터 처리 라이브러리
import pandas as pd
from pandas.io.json import json_normalize # 파이썬 딕셔너리 타입의 데이터를 판다스 데이터프레임으로 변환한다.

# 전국 또는 특정 지역의 스타벅스 매장 위치 정보를 가져와서 지도 위에 표시하기
requests로 스타벅스 매장의 위치정보를 가져와서 딕셔너리 타입으로 변환한다.  
json_normalize() 함수로 json 타입의 데이터가 변환된 파이썬 딕셔너리를 판다스 데이터프레임으로 변환한다.  
folium 모듈을 사용해서 지도를 표시하고 지도 위에 스타벅스 매장 위치를 마커로 표시한다.


In [2]:
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=3112E6S5DE'
req = requests.post(targetSite, data = {
    'ins_lat': 37.5717888, # 위도
    'ins_lng': 126.9661696, # 경도
    'p_sido_cd': '01', # 시도코드
    'p_gugun_cd':'0101', # 구군코드
    'in_biz_cd': '', # ?
    'iend': 2000, # 서버가 응답하는 최대 매장의 개수
    'set_date': '' # ?
    # in_biz_cd, set_date를 넘겨주지 않으면 서버에서 데이터를 얻어오지 못한다.
})
starbucks = req.json()

In [3]:
# pandas.io.json 모듈의 json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_normalize(딕셔너리, '데이터프레임으로 변환할 데이터가 할당된 딕셔너리의 key')
starbucks_df = json_normalize(starbucks, 'list')
print(type(starbucks_df)) # 
print(len(starbucks_df))
print(starbucks_df.columns)

<class 'pandas.core.frame.DataFrame'>
89
Index(['seq', 'sido_cd', 'sido_nm', 'gugun_cd', 'gugun_nm', 'code_order',
       'view_yn', 'store_num', 'sido', 'gugun',
       ...
       't22', 't21', 'p90', 't05', 't30', 't36', 't27', 't29', 't43', 't48'],
      dtype='object', length=129)


In [4]:
starbucks_df

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,t22,t21,p90,t05,t30,t36,t27,t29,t43,t48
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
85,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
86,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
87,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# 데이터프레임으로 변환하니 서울에 있는 스타벅스 지점은 599개고 각 지점당 129개의 컬럼으로 정보를 표시하고 있다.
# 작업에 필요한 컬럼 몇 가지를 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터프레임을 만든다.
# s_name       => 지점이름
# sido_code    => 시도코드
# sido_name    => 시도이름
# gugun_code   => 구군코드
# gugun_name   => 구군이름
# doro_address => 도로명주소
# lat          => 위도
# lot          => 경도

In [6]:
# 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 얻어올 컬럼이 1개라면 컬럼 이름만 적어도 되지만 얻어올 컬럼이 2개
# 이상일 경우 얻어올 컬럼들을 리스트로 만들어서 얻어와야 한다.
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot']]
starbucks_df_map.head()

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,역삼역,01,서울,0101,강남구,"서울특별시 강남구 테헤란로 145, 우신빌딩 1층 (역삼동)",37.50080096,127.0351577
1,역삼이마트,01,서울,0101,강남구,서울특별시 강남구 역삼로 310 (역삼동),37.499367,127.048425
2,삼성동,01,서울,0101,강남구,서울특별시 강남구 영동대로96길 12 (삼성동),37.510843366121534,127.06363342044263
3,강남논현,01,서울,0101,강남구,서울특별시 강남구 강남대로 512 (논현동),37.5078978596254,127.023338614644
4,테헤란로아남타워,01,서울,0101,강남구,서울특별시 강남구 테헤란로 311 (역삼동),37.50383,127.04487


In [7]:
# dtypes 속성으로 판다스 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
# 판다스는 문자열 데이터 타입을 object라 부른다.
starbucks_df_map.dtypes

s_name          object
sido_code       object
sido_name       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [8]:
# info() 메소드를 사용해도 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   s_name        89 non-null     object
 1   sido_code     89 non-null     object
 2   sido_name     89 non-null     object
 3   gugun_code    89 non-null     object
 4   gugun_name    89 non-null     object
 5   doro_address  89 non-null     object
 6   lat           89 non-null     object
 7   lot           89 non-null     object
dtypes: object(8)
memory usage: 5.7+ KB


In [9]:
# astype() 메소드로 위도, 경도의 데이터 타입을 object에서 float로 변환한다.
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   s_name        89 non-null     object 
 1   sido_code     89 non-null     object 
 2   sido_name     89 non-null     object 
 3   gugun_code    89 non-null     object 
 4   gugun_name    89 non-null     object 
 5   doro_address  89 non-null     object 
 6   lat           89 non-null     float64
 7   lot           89 non-null     float64
dtypes: float64(2), object(6)
memory usage: 5.7+ KB


In [10]:
# 종각점
starbucks_df_map[starbucks_df_map['s_name'] == '종각']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot


In [11]:
# 종로관철점
starbucks_df_map[starbucks_df_map['s_name'] == '종로관철 ']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot


In [12]:
# 종로구청점
starbucks_df_map[starbucks_df_map['s_name'] == '종로구청']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot


브로드 캐스팅을 이용한 불린 인덱싱

In [13]:
starbucksMap = folium.Map(location=[37.5697987, 126.9837167], zoom_start=17)
popup = folium.Popup('종각점', max_width=300)
folium.Marker(location=[37.569918, 126.984528], popup=popup).add_to(starbucksMap)
popup = folium.Popup('종로관철점', max_width=300)
folium.Marker(location=[37.569058, 126.986013], popup=popup).add_to(starbucksMap)
popup = folium.Popup('종로구청점', max_width=300)
folium.Marker(location=[37.572662, 126.981138], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap1.html')
starbucksMap